In [1]:
%run CRLBO.py 

In [2]:
# Initializations (AML)

input_file_path = r'D:\Temp\1. Non Augmented AML Incremental EWC\Actual Splits'
input_file_name = 'AML_Non_Aug_Full_Batch.csv'
output_file_path = r'D:\Temp\1. Non Augmented AML Incremental EWC\Actual Splits\40 Batches\Output'
model_files_path = r'D:\Temp\1. Non Augmented AML Incremental EWC\Actual Splits\40 Batches\Models'

Class_label = 'Class'
Splits = 40

epochs_ = 20
penalty_param = 'L2' # penalty_param = SGD, L2, CRLBO
weight_decay = 0.1 # In case of L2 penalty
fisher_scalar = 1000000
crit = nn.CrossEntropyLoss()


In [3]:
# # Initializations (Credit Default  v1 to v28)

# input_file_path = r'D:\Temp\4. Non Augmented v1 to v28\Actual Splits'
# input_file_name = 'v1_v28_Non_Aug_Full_Batch.csv'
# output_file_path = r'D:\Temp\4. Non Augmented v1 to v28\Actual Splits\40 Batches\Output'
# model_files_path = r'D:\Temp\4. Non Augmented v1 to v28\Actual Splits\40 Batches\Models'

# Class_label = 'Class'
# Splits = 40

# epochs_ = 20
# penalty_param = 'CRLBO' # penalty_param = SGD, L2, CRLBO
# weight_decay = 0.1 # In case of L2 penalty
# fisher_scalar = 1000000
# crit = nn.CrossEntropyLoss()


In [4]:
# # Initializations (Credit Mortgage)

# input_file_path = r'D:\Temp\2. Non Augmented Mortgage\Actual Splits'
# input_file_name = 'Mortgage_Non_Aug_Full_Batch.csv'
# output_file_path = r'D:\Temp\2. Non Augmented Mortgage\Actual Splits\40 Batches\Output'
# model_files_path = r'D:\Temp\2. Non Augmented Mortgage\Actual Splits\40 Batches\Models'

# Class_label = 'TARGET'
# Splits = 40

# epochs_ = 20
# penalty_param = 'CRLBO' # penalty_param = SGD, L2, CRLBO
# weight_decay = 0.1 # In case of L2 penalty
# fisher_scalar = 1000000
# crit = nn.CrossEntropyLoss()


In [5]:
# # Initializations (Paysim)

# input_file_path = r'D:\Temp\3. Non Augmented Paysim\Actual Splits'
# input_file_name = 'Paysim_Non_Aug_Full_Batch.csv'
# output_file_path = r'D:\Temp\3. Non Augmented Paysim\Actual Splits\40 Batches\Output'
# model_files_path = r'D:\Temp\3. Non Augmented Paysim\Actual Splits\40 Batches\Model'

# Class_label = 'isFraud'
# Splits = 40

# epochs_ = 20
# penalty_param = 'CRLBO' # penalty_param = SGD, L2, CRLBO
# weight_decay = 0.1 # In case of L2 penalty
# fisher_scalar = 1000000
# crit = nn.CrossEntropyLoss()


In [6]:
# Pre-processing
features = Pre_Processing(input_file_path, input_file_name, output_file_path, Class_label, Splits)

Input Shape: (3296087, 330)
Input Classes: 0    3291592
1       4495
Name: Class, dtype: int64


In [7]:
features

329

In [8]:
# Model Training and Testing
result = {} 

for i in range(0, Splits): 
    
    train_loader, test_loader, train_set, mini_batches = read_batch(i, output_file_path)
    
    train_key = 'train_batch_' + str(i)
    result[i] = train_key
    
    # Model definition
    crlbo = CramerRaoLowerBound(BaseModel(features, 10, 2), crit=crit, lr=1e-4, weight = fisher_scalar, penalty = penalty_param, weight_decay_param = weight_decay)
    
    for epochs in range(epochs_):
        for input, target in tqdm(train_loader):
            crlbo.forward_backward_update(input.float(), target.long(), penalty = penalty_param)
    
    if penalty_param == 'CRLBO':
        crlbo.register_crlbo_params(train_set, 100, mini_batches)
    
    crlbo.save(model_files_path + '\\' + penalty_param + '_batch_' + str(i) + '.h5')
    
    acc = accu(crlbo.model, test_loader)
    result[train_key] = acc
    
    # Testing on previous batches
    for j in range(i):
        train_loader, test_loader, train_set, mini_batches = read_batch(j, output_file_path)
        test_key = 'test_batch_' + str(j)
        result[j] = test_key
        
        acc = accu(crlbo.model, test_loader)
        result[train_key+'_'+test_key] = acc

100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 152.40it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 160.35it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 134.04it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 151.22it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 145.76it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 156.65it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:04<00:00, 153.38it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:03<00:00, 176.16it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:03<00:00, 176.61it/s]


100%|███████████████████████████████████████████████████████████████████████████████| 660/660 [00:03<00:00, 168.48it/s]


In [9]:
i = 0
dict_result = {}
list_result = []
for key in result:
    list_result.append(result[key])
    
    if(key == i):
        del list_result[-1]
        if(i != 0):
            list_result.append(list_result[0])
            del list_result[0]
        dict_result[i] = list_result
        list_result = []
        i= i + 1

list_result.append(list_result[0])
del list_result[0]
dict_result[i] = list_result
            

In [10]:
pd.DataFrame.from_dict(dict_result, orient='index').to_csv(model_files_path + '\\' + penalty_param + '_results.csv')

In [11]:
crlbo.model

BaseModel(
  (f1): Flatten()
  (lin1): LinearLayer(
    (lin): Linear(in_features=329, out_features=10, bias=True)
    (act): ReLU()
    (bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lin2): LinearLayer(
    (lin): Linear(in_features=10, out_features=10, bias=True)
    (act): ReLU()
    (bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lin3): Linear(in_features=10, out_features=2, bias=True)
)

In [12]:
# End of Script